In [2]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim
import math
from torchsummary import summary

In [3]:
import os
import pandas as pd
import traceback

In [98]:
def dataloaderV2(dossier, etat):
    # on part d'un dossier racine où chacun des dossiers contenant les données est
    # initialise data = tableau final
    data = []
    # initialise le di;ctionnaire qui va stocker toutes les valeurs de session et class par individu
    
    #on va parcourir tous les dossier du premier dossier
    for dos in os.listdir(dossier):
        #on met à jour le path pour pouvoir parcourir le sous dossier
        dos_path = os.path.join(dossier, dos)
        
        sorted_files = sorted(os.listdir(dos_path))
        
        #on vérifie que c'est bien un dossier
        if os.path.isdir(dos_path):
            # on va parcourir les fichiers contenant les datas
            n_va = 0
            for fichier in sorted_files:
                #on initialise l'indice de VA qui va vous permettre de parcourir le data final par colonnes
                #vérifie que c'est bien un fichier csv
                if fichier.endswith(".csv") and fichier.split("_")[0].lower()== etat:
                    #print("fichier", fichier) #okay va dans tous les fichiers
                    #met à jour le path pour pouvoir ouvrir le fichier
                    filepath = os.path.join(dos_path, fichier)
                    try:
                        df = pd.read_csv(filepath, header = 0, low_memory=False)
                    except Exception as e:
                        print(f"Error reading file '{va}': {e}")
                        
                    for indiv in range(len(data)):
                        # Expand this individual's list if it doesn't have enough slots
                        while len(data[indiv]) <= n_va:
                            required_length = n_va + 1
                            current_length = len(data[indiv])
                            additional_slots_needed = required_length - current_length
                            if additional_slots_needed > 0:
                                data[indiv].extend([None] * additional_slots_needed)
                            
                    # on va otérer sur toutes les datas de ce fichier, 
                    # avec num indiv indice de 0 ua bout, index = index de lignes et row la data
                    for indiv, (index, row) in enumerate(df.iterrows()):
                        try:
                            #print(f"before indiv: {indiv}, n_va: {n_va}, len(data[indiv]): {len(data[indiv])}")
                            # Ensure the sublist exists and is of the correct length
                            while len(data) <= indiv:
                                #print("data avant ajustement pas d'indiv : ", data) okay les datas sont correctes
                                # Extend data list with empty lists as needed
                                #data.extend([[] for _ in range(indiv - len(data) + 1)])
                                data.extend([[]] * (indiv - len(data) + 1))
                                # Diagnostic prints to understand the structure just before the error occurs
                                #print(f"Attempting assignment to data[{indiv}][{n_va}].")
                                #print(f"len(data): {len(data)}, len(data[{indiv}]): {len(data[indiv])}.")

                                #print("data avant ajustement pas d'indiv : ", data)okay les datas sont correctes
                            while len(data[indiv]) <= n_va:
                                #print("rien dans les indiv = pas de va AVANT : ", data) okay les datas sont correctes
                                # Extend the sublist with None values as placeholders
                                #data[indiv].extend([None for _ in range(n_va - len(data[indiv]) + 1)])
                                data[indiv].append([])    
                                #data[indiv].append(None)
                            #print(f"after indiv: {indiv}, n_va: {n_va}, len(data[indiv]): {len(data[indiv])}")
                            
                                #print("rien dans les indiv = pas de va APRES : ", data)
                            # Now safe to assign the value
                            #print(f"Right before assignment to data[{indiv}][{n_va}]: len(data): {len(data)}, len(data[{indiv}]): {len(data[indiv])}, len(data[{indiv}][{n_va}]): 'Will be set'")

                           
                            # Extract session and class if already present
                            if len(data[indiv]) >= 2:
                                #print("session class deja la AVANT : ", data[indiv])# okay les datas sont correctes
                                session_data, class_data = data[indiv][-2:]  # Extract existing session and class
                                data[indiv] = data[indiv][:-2]  # Remove session and class from the current list
                                #print("session class deja la APRES : ", data) okay les datas sont correctes
                                #print("session class deja la APRES : ", data[indiv]) okay les datas sont correctes
                                #print("session : ", session_data, " class : ", class_data) okay les datas sont correctes
                            else:
                                #print("recup session class depuis datas")
                                session_data, class_data = row.iloc[-2], row.iloc[-1]  # Get new session and class
                                #print("session : ", session_data, " class : ", class_data) print("session : ", session_data, " class : ", class_data

                            # For each individual, append the new row data, excluding session and class
                            #row_data = row.iloc[:-2].tolist()
                            #data[indiv].extend(row_data)  # Append the new row data
                            
                           # print("data avant append nouvelle va : ", data[indiv][n_va]) okay les datas sont correctes
                            data[indiv].append(row.iloc[:-2].tolist())
                            #print("data apres append nouvelle va : ", data[indiv][n_va]) okay les datas sont correctes
                            # Re-append the session and class data at the end
                            #print("data avant append nouvelle va append session/class: ", data[indiv]) okay les datas sont correctes
                            data[indiv].append([session_data])
                            #print("data apres append nouvelle va append session: ", data[indiv]) okay les datas sont correctes
                            data[indiv].append([class_data])
                            #print("data apres append nouvelle va append class: ", data[indiv]) okay les datas sont correctes
                            
                        #except (IndexError, TypeError):
                        except (IndexError, TypeError) as e:
                            print(f"error: {type(e).name}")
                            #if isinstance(row.iloc[:-2], pd.Series):
                             #   data.append([row.iloc[:-2].tolist()])
                            #traceback.print_exc()
                            # Properly append a new sublist if catching an IndexError before this logic
                            #if isinstance(row.iloc[:-2], pd.Series):
                            #    data.append([row.iloc[:-2].tolist()])
                            #    print("error")
                            #else:
                                # Handle unexpected data format
                             #   data.append([None])
                            
                    n_va +=1

    return data 

def nettoyage(dat):
    for i in range(len(dat)):
        dat[i].pop(0)
        dat[i].pop(-2)
        dat[i].pop(-1)
    return dat

directory_path = "./RAW"
data_n = dataloaderV2(directory_path, "normal")
data_f = dataloaderV2(directory_path, "fatigue")

data_n = nettoyage(data_n)
data_f = nettoyage(data_f)

data_n0 = []
data_n1 = []
split_index = len(data_n[0]) // 2
for i in range(len(data_n)):
    data_n0.append(data_n[i][:split_index])
    data_n1.append(data_n[i][split_index:])
    
data_n = data_n0+data_n1

data_f0 = []
data_f1 = []
split_index = len(data_f[0]) // 2
for i in range(len(data_f)):
    data_f0.append(data_f[i][:split_index])
    data_f1.append(data_f[i][split_index:])
    
data_f = data_f0+data_f1

for i in range(len(data_n)):
    data_n[i].append([0])
for i in range(len(data_f)):
    data_f[i].append([1])
    
data = data_f + data_n

print("LEN DATA")
print(len(data))
print("LEN DATA[0]")
print(len(data[0]))
print("LEN COntainer")
for i in range(len(data[0])):
    print(len(data[0][i]))
print(data[2000][-1])

def extract_values(input_list):
    values = []
    for item in input_list:
        if isinstance(item, (list, tuple)):
            values.extend(extract_values(item))
        else:
            values.append(item)
    return values


new_data = []
for i in range(len(data)):
    new_data.append(extract_values(data[i]))
print(len(new_data))
print(len(new_data))


LEN DATA
4218
LEN DATA[0]
11
LEN COntainer
3840
3840
3840
7680
480
120
120
120
120
120
1
[1]
4218
4218


In [124]:
class CustomDataset(Dataset):
    def __init__(self, data):
        print(len(data))
        self.data = pd.DataFrame(data)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values
        print(len(self.features))#j'ai fait ma modif ici
        print(len(self.targets))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target

dataset = CustomDataset(new_data)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)

4218
4218
4218


In [125]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 31202
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 16384)
        self.fc2 = nn.Linear(16384, 4096)
        self.fc3 = nn.Linear(4096, 1024)
        self.fc4 = nn.Linear(1024, 256)
        self.fc5 = nn.Linear(256, 64)
        self.fc6 = nn.Linear(64, self.out_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
        
    
    def forward(self, x):
        a1 = self.relu(self.fc1(x))
        a2 = self.relu(self.fc2(a1))
        a3 = self.relu(self.fc3(a2))
        a4 = self.relu(self.fc4(a3))
        a5 = self.relu(self.fc5(a4))
        return self.fc6(a5)
        
model = MLP()
#for i in range(ModelNum):
#    #summary(model[i], (5,))
#    if any(p.is_cuda for p in model[i].parameters()):
#        print("Model is on CUDA")
#    else:
#        print("Model is on CPU")

# Summarize models

In [126]:
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i + 1) % 20 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,    20] loss: nan
[1,    40] loss: nan
[1,    60] loss: nan


KeyboardInterrupt: 